# Base de données NoSQL - MongoDB

## **Installation MongoDB**

Voici deux méthodes pour installer mongo du plus simple au plus complexe (à utiliser en fonction de votre préférence ou en cas d'echec d'une méthode)  
**1 - avec docker**  
a) Commencons par installer docker soit avec :  
sudo snap install docker  
ou avec apt:  
apt install docker.io  
b) Ensuite, créer un repertoire ou seront stocker les données de la MongoDB, par exemple  
mkdir -p ~/data-mongodb  
c) il suffit ensuite d'installer mongo avec la suivante suivante:  
docker run -d -p 27017:27017 -v ~/src/data/mongo-docker:/data/db --name mongodb mongo:4.2  
ici, docker se charge d'installer tout le necessaire et de placer le tout dans un conteneur isolé.  
d) Si l'installation a fonctionné, vous pourrez renter dans un interpreteur mongodb avec la commande suivante:  
docker exec -it mongodb mongo  
**2 - avec apt**   
voir le tuto pour debian ou ubuntu sur la doc mongo.

## **Prise en main MongoDB**  
https://docs.mongodb.com/manual/reference/method/js-database/  
https://riptutorial.com/fr/mongodb/example/30003/creer-une-collection

## **Restart mongoDB et commandes de base**

sudo docker restart mongodb  
sudo docker exec -it mongodb mongo  

**Commandes de base**  
show dbs  <span style="color: #1b019b">Montre les tables disponibles</span>  
use test  <span style="color: #1b019b">Utilise la base de donnée "test", la crée si elle n'esxiste pas</span>  
db.offres_emploi.find() <span style="color: #1b019b">Dans la base en cours d'utilisation,affiche les données de la collection "offres_emploi"</span>  
  
https://docs.mongodb.com/manual/reference/mongo-shell/  
 

## **Interface python/mongodb**

Installation de la librairie python:  
pip install --user pymongo  
Installation dans l'environnement anaconda:  
conda install pymongo

#### **Se connecter à la base, ajout de donnée, affichage des données contenues dans la collection**  

In [1]:
import pymongo
from pymongo import MongoClient
import pprint

client = MongoClient ('localhost',27017)
db = client.test

# Nouvel enregistrement:
nd={"title":"Big Data Developer - II","description": "NY-Garden City, Location: Garden City, NY Description: Our client is currently seeking a Big Data Developer - II Title: Big Data Developer Location: Garden City, NY Duration: 12+ Months What you?ll be doing. You will be part of a new Data Science team, ensuring that innovation drives continuous business improvements, bringing more profitable growth and an even better customer experience. As a Data Analytics/Predi",\
    "link": "http://jobview.monster.com/Big-Data-Developer-II-Job-Garden-City-NY-US-221152772.aspx",\
    "guid":221152772,"pubDate":"Thu, 12 Nov 2020 00:06:54 GMT"}

#print(nd)
#db.offres_emploi.insert_one(nd);

# Affichage avec mise en forme
pprint.pprint(list(db.offres_emploi.find()))



[{'_id': ObjectId('5fad3222423ac3bad9f64efa'),
  'description': 'FL-Tampa, Our client, one of the largest financial services '
                 'firms, is seeking a Big Data Developer Location: Tampa, FL '
                 'Position Type: Contract Job Responsibilities: -Gather '
                 'various data points from multiple sources within and load to '
                 'big data Hadoop solution. -Transform data into usable form '
                 'per user requirements. -Implement proper controls to protect '
                 'integrity of data and limit use of sensitive data to una',
  'guid': 221978899.0,
  'link': 'http://jobview.monster.com/Big-Data-Developer-Job-Tampa-FL-US-221978899.aspx',
  'pubDate': 'Thu, 12 Nov 2020 02:31:23 GMT',
  'title': 'Big Data Developer'},
 {'_id': ObjectId('5fad4adbeb7b7b482b8413fd'), 'toto': 'titi'},
 {'_id': ObjectId('5fad4b57eb7b7b482b841400'),
  'description': 'NY-Garden City, Location: Garden City, NY Description: Our '
                 'c

## **Scrapy**

### **Présentation de scrapy**

Scrapy est un framework open source et collaborative pour l'extraction de données sur le net.  
Scrapy est un outil créer spécifiquement pour effectuer des requêtes, scraper et sauvegarder des données sur le web. Il se suffit à lui même pour construire un projet de webscraping robuste.  

<span style="color: #0000FF">Exploiter le site:</span>  
https://medium.com/@alexandrewrg/scraper-les-donn%C3%A9es-de-plusieurs-pages-avec-scrapy-2e076ac7dc09


### **Bot de scrapy conçu par Adrien**

**Run the spider et création d'un fichier csv**  
scrapy runspider flashbot.py -o offres_emploi_scrapy.csv  


**For local testing**  
get the RSS page locally:  
wget rss.jobsearch.monster.com/rssquery.ashx -O index.xml  
  
Replace the value variable start_urls in the flashbot by the **full path** (obtained with pwd).  
Full path commence par un '/', d'ou les 3 '/' dans 'file:'  
  
<span style="color: #FF0000">Étude du code:  
yield, xpath,...</span>  


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Nov 13 15:40:18 2020

@author: Adrien D.
"""

# -*- coding: utf-8 -*-
import scrapy
from scrapy import Request

from pprint import pprint


class FlashbotSpider(scrapy.Spider):
    name = 'flashbot'
    allowed_domains = ['rss.jobsearch.monster.com']

    # Start the crawler at this URLs
    start_urls = ['file:///home/jpphi/Documents/brief/brief3-bddNosql/rssquery.xml']
    #start_urls = ['http://localhost:8888/view/Documents/brief/brief3-bddNosql/rssquery.ashx.xml']
    #start_urls = ['http://rss.jobsearch.monster.com/rssquery.ashx?q={query}']

    thesaurus = ["machine learning", "machine", "learning", "big data", "big", "data"]

    LOG_LEVEL = "INFO"

    def parse(self, response):

        # We stat with this url
        url = self.start_urls[0]

        # Build and send a request for each word of the thesaurus
        for query in self.thesaurus:
            target = url.format(query=query)
            print("fetching the URL: %s" % target)
            if target.startswith("file://"):
                r = Request(target, callback=self.scrapit, dont_filter=True)
            else:
                r = Request(target, callback=self.scrapit)
            r.meta['query'] = query
            yield r

    def scrapit(self, response):
        query = response.meta["query"]

        # Base item with query used to this response
        item = {"query": query}
        print(query, response)

        # Scrap the data
        for doc in response.xpath("//item"):
            item["title"] = doc.xpath("title/text()").extract()
            item["description"] = doc.xpath("description/text()").extract()
            item["link"] = doc.xpath("link/text()").extract()
            item["pubDate"] = doc.xpath("pubDate/text()").extract()
            item["guid"] = doc.xpath("guid/text()").extract()
            #pprint(item, indent=2)
            print("item scraped:", item["title"])
            yield item
            
runspider

### **Installation de Scrapy**

## Ressources

* https://www.digitalocean.com/community/tutorials/how-to-make-a-web-application-using-flask-in-python-3-fr
* https://openclassrooms.com/fr/courses/4425066-concevez-un-site-avec-flask  
mongodb:  
* https://www.mongodb.com/
* https://docs.mongodb.com/  
scrapy:  
* https://scrapy.org
* https://docs.scrapy.org

